인터넷에서 다운 받은 json파일을 
특정 클래스만 가진 json파일로 변환하는 코드

In [2]:
import os
import json

# 입력 및 출력 폴더 경로 설정
input_folder = "/home/ted/dev_ws/deeplearing_pj/mask/tolabel/all/json"    # JSON 파일들이 저장된 폴더 경로
output_folder = "/home/ted/dev_ws/deeplearing_pj/0316/1_converted_json"  # 변환된 JSON 파일들을 저장할 폴더 경로

# 남길 클래스 목록 (필요에 따라 이 리스트에 원하는 class_ID를 추가하면 됩니다)
classes_to_keep = ["WO-03", "SO-25", "SO-28", "SO-20", "SO-40"]

# 출력 폴더가 없으면 생성
os.makedirs(output_folder, exist_ok=True)

# 입력 폴더 내의 모든 JSON 파일 처리
for filename in os.listdir(input_folder):
    if filename.endswith(".json"):
        input_path = os.path.join(input_folder, filename)
        
        # JSON 파일 읽기
        with open(input_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # "Learning_Data_Info." 내의 "Annotations" 리스트 필터링:
        # 남길 클래스 목록에 포함된 항목만 남김
        annotations = data.get("Learning_Data_Info.", {}).get("Annotations", [])
        filtered_annotations = [ann for ann in annotations if ann.get("class_ID") in classes_to_keep]
        
        if "Learning_Data_Info." in data:
            data["Learning_Data_Info."]["Annotations"] = filtered_annotations
        
        # 결과를 출력 폴더에 저장
        output_path = os.path.join(output_folder, filename)
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        
        print(f"{filename} 변환 완료")

print("모든 파일 변환이 완료되었습니다.")

H-220921_D31_Y-39_002_0111.json 변환 완료
H-221011_D18_Y-39_002_0210.json 변환 완료
H-220919_D15_Y-34_010_0067.json 변환 완료
H-220926_A35_Y-10_006_0007.json 변환 완료
H-220930_A35_Y-09_004_0184.json 변환 완료
H-220926_A35_Y-10_006_0097.json 변환 완료
H-220829_D31_Y-39_002_0012.json 변환 완료
H-220926_A35_Y-09_007_0021.json 변환 완료
H-220930_A35_Y-09_004_0164.json 변환 완료
H-220930_A35_Y-09_004_0075.json 변환 완료
H-221011_D18_Y-39_002_0280.json 변환 완료
H-220830_D31_Y-34_012_0029.json 변환 완료
H-220829_D31_Y-39_002_0051.json 변환 완료
H-220926_A35_Y-09_008_0099.json 변환 완료
H-220926_A35_Y-09_004_0028.json 변환 완료
H-220829_D31_Y-39_003_0047.json 변환 완료
H-220930_A35_Y-09_004_0098.json 변환 완료
H-220707_A32_Y-10_030_0029.json 변환 완료
H-220829_D31_Y-39_003_0051.json 변환 완료
H-220901_D18_Y-34_005_0030.json 변환 완료
H-220926_A35_Y-10_005_0174.json 변환 완료
H-220819_D19_Y-39_005_0080.json 변환 완료
H-221011_D18_Y-39_003_0035.json 변환 완료
H-220930_A35_Y-09_004_0099.json 변환 완료
H-220921_D31_Y-39_002_0081.json 변환 완료
H-220919_D15_Y-34_002_0035.json 변환 완료
H-220926_A35

인터넷에서 다운 받은 json파일을 
내가 직접 작성한 labelme형식으로 변환시키는 코드

In [3]:
import os
import json
import base64

# 변환할 인터넷 JSON 파일들이 있는 폴더와 변환된 파일을 저장할 폴더, 그리고 이미지 파일들이 보관된 폴더를 지정합니다.
input_dir = "/home/ted/dev_ws/deeplearing_pj/0316/1_converted_json"      # 인터넷 JSON 파일 폴더 경로로 변경
output_dir = "/home/ted/dev_ws/deeplearing_pj/0316/2_converted_json"      # 변환된 LabelMe JSON 파일을 저장할 폴더 경로로 변경
image_dir = "/home/ted/dev_ws/deeplearing_pj/mask/tolabel/all/image"               # 실제 이미지 파일이 보관된 폴더 경로로 변경

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def parse_resolution(resolution_str):
    # 해상도 문자열 예: "1920, 1080" 또는 "1920x1080"
    if "," in resolution_str:
        parts = resolution_str.split(",")
    elif "x" in resolution_str.lower():
        parts = resolution_str.lower().split("x")
    else:
        return None, None
    try:
        width = int(parts[0].strip())
        height = int(parts[1].strip())
        return width, height
    except:
        return None, None

def load_image_as_base64(image_path):
    if os.path.exists(image_path):
        with open(image_path, "rb") as f:
            img_data = f.read()
        return base64.b64encode(img_data).decode("utf-8")
    else:
        return ""

def convert_internet_json_to_labelme(in_data):
    # LabelMe 기본 구조 생성
    labelme_data = {
        "version": "4.5.6",
        "flags": {},
        "shapes": [],
        "imagePath": "",
        "imageData": "",
        "imageHeight": None,
        "imageWidth": None
    }
    
    # 이미지 정보: Source_Data_Info.와 Raw_Data_Info. 사용
    source_info = in_data.get("Source_Data_Info.", {})
    raw_info = in_data.get("Raw_Data_Info.", {})
    src_data = source_info.get("Source_Data", "unknown")
    file_ext = source_info.get("File_extension", "jpg")
    image_filename = f"{src_data}.{file_ext}"
    labelme_data["imagePath"] = image_filename

    # image_dir에 실제 이미지 파일이 존재한다면 base64 인코딩한 값을 채워넣습니다.
    image_path = os.path.join(image_dir, image_filename)
    labelme_data["imageData"] = load_image_as_base64(image_path)
    
    # 해상도 파싱
    resolution = raw_info.get("Resolution", "")
    width, height = parse_resolution(resolution)
    labelme_data["imageWidth"] = width
    labelme_data["imageHeight"] = height
    
    # Learning_Data_Info.에서 어노테이션 정보 처리
    annotations = in_data.get("Learning_Data_Info.", {}).get("Annotations", [])
    for ann in annotations:
        shape = {}
        shape["label"] = ann.get("class_ID", "")
        ann_type = ann.get("type", "").lower()
        
        if ann_type == "polygon":
            points_flat = ann.get("value", [])
            if len(points_flat) % 2 != 0:
                continue
            points = [[points_flat[i], points_flat[i+1]] for i in range(0, len(points_flat), 2)]
            shape["points"] = points
            shape["shape_type"] = "polygon"
        elif ann_type == "bbox":
            bbox = ann.get("value", [])
            if len(bbox) == 4:
                x_min, y_min, x_max, y_max = bbox
                points = [
                    [x_min, y_min],
                    [x_max, y_min],
                    [x_max, y_max],
                    [x_min, y_max]
                ]
                shape["points"] = points
                shape["shape_type"] = "polygon"
            else:
                continue
        else:
            continue
        
        shape["group_id"] = None
        shape["flags"] = {}
        labelme_data["shapes"].append(shape)
    
    return labelme_data

# 폴더 내의 모든 JSON 파일을 변환
for filename in os.listdir(input_dir):
    if filename.endswith(".json"):
        input_path = os.path.join(input_dir, filename)
        with open(input_path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
            except Exception as e:
                print(f"Error loading {filename}: {e}")
                continue
            
        labelme_json = convert_internet_json_to_labelme(data)
        
        output_path = os.path.join(output_dir, filename)
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(labelme_json, f, indent=4, ensure_ascii=False)
        
        print(f"Converted {filename} -> {output_path}")


Converted H-220921_D31_Y-39_002_0111.json -> /home/ted/dev_ws/deeplearing_pj/0316/2_converted_json/H-220921_D31_Y-39_002_0111.json
Converted H-221011_D18_Y-39_002_0210.json -> /home/ted/dev_ws/deeplearing_pj/0316/2_converted_json/H-221011_D18_Y-39_002_0210.json
Converted H-220919_D15_Y-34_010_0067.json -> /home/ted/dev_ws/deeplearing_pj/0316/2_converted_json/H-220919_D15_Y-34_010_0067.json
Converted H-220926_A35_Y-10_006_0007.json -> /home/ted/dev_ws/deeplearing_pj/0316/2_converted_json/H-220926_A35_Y-10_006_0007.json
Converted H-220930_A35_Y-09_004_0184.json -> /home/ted/dev_ws/deeplearing_pj/0316/2_converted_json/H-220930_A35_Y-09_004_0184.json
Converted H-220926_A35_Y-10_006_0097.json -> /home/ted/dev_ws/deeplearing_pj/0316/2_converted_json/H-220926_A35_Y-10_006_0097.json
Converted H-220829_D31_Y-39_002_0012.json -> /home/ted/dev_ws/deeplearing_pj/0316/2_converted_json/H-220829_D31_Y-39_002_0012.json
Converted H-220926_A35_Y-09_007_0021.json -> /home/ted/dev_ws/deeplearing_pj/0316/2

특정폴더에 클래스별 몇개의 사진이 있는지 확이하는 코드

In [1]:
import os
import json

# JSON 파일들이 저장된 폴더 경로 (실제 경로로 수정)
folder_path = "/home/ted/dev_ws/deeplearing_pj/0316/last_json/"

# 집계할 클래스 목록
classes_to_count = ["WO-03", "SO-25", "SO-28", "SO-20", "SO-40"]

# 각 클래스별 카운트를 저장할 딕셔너리 초기화
class_counts = {cls: 0 for cls in classes_to_count}

# 폴더 내의 모든 JSON 파일에 대해 처리
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # JSON 파일 형식에 따라 "shapes" 리스트를 확인 (예제에서는 LabelMe 형식 사용)
        shapes = data.get("shapes", [])
        for shape in shapes:
            label = shape.get("label")
            if label in class_counts:
                class_counts[label] += 1

# 결과 출력
for cls, count in class_counts.items():
    print(f"{cls}: {count}")


WO-03: 0
SO-25: 0
SO-28: 0
SO-20: 0
SO-40: 1600


In [ ]:
labelme2yolo --json_dir /home/ted/dev_ws/deeplearing_pj/0316/last_json/ --output_format polygon

In [2]:
from ultralytics import YOLO

# 사전 학습된 segmentation 모델 로드
model = YOLO('yolov8n-seg.pt')

# 학습 실행
results = model.train(
    data='/home/ted/dev_ws/deeplearing_pj/0316/2_converted_json/YOLODataset/dataset.yaml',  # 데이터 설정 파일 경로
    epochs=50,    # 학습 에폭 수
    imgsz=640,     # 입력 이미지 크기
    task='segment' # segmentation 작업 지정
)

# 학습 결과 출력
print(results)


New https://pypi.org/project/ultralytics/8.3.92 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.82 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 2060, 5920MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/home/ted/dev_ws/deeplearing_pj/0316/2_converted_json/YOLODataset/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False,

train: Scanning /home/ted/dev_ws/deeplearing_pj/0316/2_converted_json/YOLODataset/labels/train... 5348 images, 1 backgrounds, 0 corrupt: 100%|██████████| 5348/5348 [00:03<00:00, 1753.83it/s]


train: New cache created: /home/ted/dev_ws/deeplearing_pj/0316/2_converted_json/YOLODataset/labels/train.cache


val: Scanning /home/ted/dev_ws/deeplearing_pj/0316/2_converted_json/YOLODataset/labels/val... 1337 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1337/1337 [00:02<00:00, 498.74it/s]


val: New cache created: /home/ted/dev_ws/deeplearing_pj/0316/2_converted_json/YOLODataset/labels/val.cache
Plotting labels to runs/segment/train9/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/segment/train9
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50      2.57G     0.3675     0.6439     0.2137     0.8079          5        640: 100%|██████████| 335/335 [01:12<00:00,  4.60it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:11<00:00,  3.78it/s]

                   all       1337       2009      0.978      0.994      0.992      0.884      0.975      0.983      0.983      0.786



50 epochs completed in 1.145 hours.
Optimizer stripped from runs/segment/train9/weights/last.pt, 6.8MB
Optimizer stripped from runs/segment/train9/weights/best.pt, 6.8MB

Validating runs/segment/train9/weights/best.pt...
Ultralytics 8.3.82 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 2060, 5920MiB)
YOLOv8n-seg summary (fused): 85 layers, 3,259,429 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:12<00:00,  3.33it/s]


                   all       1337       2009      0.976      0.997      0.992      0.883      0.972      0.985      0.985      0.787
                 WO-03        646        646      0.999          1      0.995      0.993      0.999          1      0.995      0.896
                 SO-40        627        627      0.991          1      0.995       0.93      0.991          1      0.995      0.808
                 SO-20        312        312      0.988          1      0.995      0.955      0.988          1      0.995      0.919
                 SO-25         98         98      0.994          1      0.995      0.949      0.995          1      0.995      0.793
                 WO-24         58         58      0.975          1      0.987      0.915      0.976          1      0.987      0.854
                 WO-25         54         54      0.914      0.979      0.984      0.573      0.874      0.901       0.93      0.482
                 SO-28        214        214      0.968          1   

In [3]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# 모델 로드 (훈련된 모델 경로 지정)
model = YOLO('/home/ted/dev_ws/deeplearing_pj/runs/segment/train9/weights/best.pt')  # best.pt가 저장된 위치

# 테스트할 이미지 경로 지정
image_path = '/home/ted/Downloads/1126495-1-2.jpg'  # 테스트할 이미지 파일

# 이미지 예측
results = model.predict(image_path, save=True, show=True)  # 결과 저장 및 출력

# 결과 이미지 표시 (OpenCV 사용)
for result in results:
    img = result.plot()  # 검출된 객체를 포함한 이미지 반환
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()
OpenCV(4.11.0) /io/opencv/modules/highgui/src/window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


image 1/1 /home/ted/Downloads/1126495-1-2.jpg: 640x640 2 WO-03s, 7.8ms
Speed: 1.8ms preprocess, 7.8ms inference, 19.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/segment/predict4


<Figure size 1000x1000 with 1 Axes>